In [1]:
# %load_ext autoreload
# %autoreload 2
from pcfg import PCFG
from pcsg import PCSG, from_pcsg_string
import numpy as np
import pickle
import os
import random
import math
import time


from utils_grammar import (
    get_grammar_string,
    compute_random_guess_metric,
    random_sentence_generator,
    generate_similar_sequences,
    get_subgrammar_string,
    get_grammatical_sentences,
    train_test_split,
    compute_terminal_freq,
    get_perturbed_grammar,
    to_latex_equation,
    get_nongrammatical_sentences_from_perturbed_grammar
)
from example_grammar import grammar_details_dict
from utils_plot_grammar import plot_nonterminal_map, plot_nonterminal_map_with_edit
from tqdm import tqdm


import sys
sys.path.append("../")
from read_output.utils_plot import save_image_template


def get_stat(sequences):
    try:
        len_sequences = [len(sequence) for sequence in sequences]
        print(f"Number of Sequences: {len(sequences)}")
        print(f"Unique Sequences: {len(set(sequences))}")
        print(f"Max length: {max(len_sequences)}")
        print(f"Min length: {min(len_sequences)}")
        print(f"Mean length: {np.mean(len_sequences)}")
        print(f"Std length: {np.std(len_sequences)}")
        result = {
            "num sequences": len(sequences),
            "unique sequences": len(set(sequences)),
            "max": max(len_sequences),
            "min": min(len_sequences),
            "mean": np.mean(len_sequences),
            "std": np.std(len_sequences)
        }
        return result
    except Exception as e:
        print(e)
        return None


In [2]:
show_image = False

# context-free grammar

# grammar_name = "pcfg_one_character_missing"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_latin"
# grammar_name = "pcfg_cfg3b_disjoint_terminals"


# grammar_name = "pcfg_cfg3b_disjoint_terminals_one_rule_different"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_two_rules_different"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_three_rules_different"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_four_rules_different"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_five_rules_different"


# grammar_name = "pcfg_cfg3b_disjoint_terminals_leaf_0.55"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_leaf_0.60"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_leaf_0.70"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_leaf_0.80"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_leaf_0.90"


# grammar_name = "pcfg_cfg3b_disjoint_terminals_all_rules_0.55"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_all_rules_0.60"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_all_rules_0.70"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_all_rules_0.80"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_all_rules_0.90"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_all_rules_0.95"


# grammar_name = "pcfg_cfg3b_disjoint_terminals_skewed_prob"
# grammar_name = "pcfg_balanced_parenthesis"
# grammar_name = "pcfg_reverse_string"
# grammar_name = "pcfg_cfg3b_disjoint_terminals_skewed_prob"

# grammar_name = "pcfg_4_3_1_2_3_4_5_6_7_8_9_latin"
grammar_name = "pcfg_4_3_1_2_3_4_5_6_7_8_9"
# grammar_name = "pcfg_4_3_1_2_3_4_5_6_7_8_9_skewed_0.5"
# grammar_name = "pcfg_4_3_1_2_3_4_5_6_7_8_9_skewed_0.8"
# grammar_name = "pcfg_4_3_1_2_3_4_5_6_7_8_9_skewed_0.95"

# grammar_name = "pcfg_4_3_1_2_3_4_5_6_7_8_9_eq_len_uniform_prob"
# grammar_name = "pcfg_4_3_1_2_3_4_5_6_7_8_9_eq_len_skewed_prob"
# grammar_name = "pcfg_4_3_1_2_3_4_5_6_7_8_9_eq_len_skewed_prob_0.80"

# grammar_name = "pcfg_4_3_1_2_3_4_5_6_7_8_9_eq_len_all_rules_skewed_prob"

# grammar_name = "pcfg_cfg3b_eq_len_uniform_prob"
# grammar_name = "pcfg_cfg3b_eq_len_skewed_prob"
# grammar_name = "pcfg_cfg3b_eq_len_skewed_prob_0.75"
# grammar_name = "pcfg_cfg3b_eq_len_skewed_prob_0.90"


# grammar_name = "pcfg_example_regular"
# grammar_name = "pcfg_example_context_free"
# grammar_name = "pcfg_example_context_free"

# grammar_name = "pcfg_max-depth_3_max-breadth_3_rules_2_skewness_-1_alphabet_0-1-2-3-4-5-6-7-8-9"




# regular_grammar

# grammar_name = "preg_9_5_4_2_4_1_1"
# grammar_name = "preg_9_10_4_2_4_1_1"
# grammar_name = "preg_9_20_4_2_4_1_1"
# grammar_name = "preg_9_30_4_2_4_1_1"
# grammar_name = "preg_9_40_4_2_4_1_1"
# grammar_name = "preg_9_40_4_2_4_1_1_1"

# grammar_name = "preg_alphabet_2"
# grammar_name = "preg_alphabet_7"
# grammar_name = "preg_alphabet_26"

# grammar_name = "preg_numeral_2"
# grammar_name = "preg_numeral_7"
# grammar_name = "preg_numeral_10"


# grammar_name = "preg_alphabet_combined"
# grammar_name = "preg_alphabet_combined_skewed_prob"



# context-sensitive grammar

# grammar_name = "pcsg_csg3b_disjoint_terminals_A8_left"
# grammar_name = "pcsg_csg3b_disjoint_terminals_A8_right"



# grammar_name = "pcfg_double-branch_max-depth_4_max-breadth_3_alphabet_1-2-3-4-5-6-7-8-9"
# grammar_name = "pcfg_max-depth_16_max-breadth_2_rules_2_skewness_0_alphabet_0-1-2-3-4"



# standard_name

# grammar_name = "pcfg_double-branch_max-depth_4_max-breadth_3_rules_3_skewness_2_alphabet_1-2-3-4-5-6-7-8-9"
# grammar_name = "pfsa_states_2_symbols_3_index_0_alphabet_0-1"
# grammar_name = "pcfg_max-depth_3_max-breadth_3_rules_3_skewness_3_alphabet_0-1-2-3-4-5-6-7-8-9"



# import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument("--grammar_name", type=str, default="pcfg_4_5_2_10_latin", help="Grammar name")
# args = parser.parse_args()
# grammar_name = args.grammar_name


pfsa_object = {}

if grammar_name.startswith("pfsa"):
    pfsa_object = get_grammar_string(grammar_name)
    if pfsa_object is None:
        raise ValueError(grammar_name)
    else:
        assert isinstance(pfsa_object, dict)
        grammar_string = pfsa_object['grammar_string']
        if pfsa_object['entropy_analytic'] > 1000:
            raise ValueError(grammar_name)
        
else:
    grammar_string = get_grammar_string(grammar_name)



print(grammar_string)
if grammar_name.startswith("pcfg") or grammar_name.startswith("preg") or grammar_name.startswith("pfsa"):
    grammar = PCFG.fromstring(grammar_string)
elif grammar_name.startswith("pcsg"):
    grammar = from_pcsg_string(grammar_string)
else:
    grammar = None
print(f"Random guess loss: {compute_random_guess_metric(grammar)}")



# LaTeX print
# print(grammar_name.replace("_", " "))
color_dict = {
    "S" : "\\textcolor{red}",
    "A" : "\\textcolor{red}",
    "[" : "\\textcolor{blue}",
    "-" : "\\textcolor{black}",
    "B" : "\\textcolor{red}",
    "C" : "\\textcolor{red}",
    "E" : "\\textcolor{red}",
    "T" : "\\textcolor{red}",
}



From manually defined grammar

        S -> S_5 [1]
        S_5 -> B_4 C_1_1 E_4 T_1_1 [0.25]
        S_5 -> B_4 C_1_2 E_4 T_1_2 [0.25]
        S_5 -> B_4 C_1_3 E_4 T_1_3 [0.25]
        S_5 -> B_4 C_1_4 E_4 T_1_4 [0.25]
        B_4 -> B_3 [0.3333]
        B_4 -> B_3 B_3 B_3 [0.3333]
        B_4 -> B_3 B_3 [0.3333]
        B_3 -> B_2 [0.3333]
        B_3 -> B_2 [0.3333]
        B_3 -> B_2 B_2 [0.3333]
        B_2 -> B_1 [0.3333]
        B_2 -> B_1 [0.3333]
        B_2 -> B_1 B_1 B_1 [0.3333]
        B_1 -> '2' '9' '3' [0.3333]
        B_1 -> '9' '6' '1' [0.3333]
        B_1 -> '1' '8' '6' '2' [0.3333]
        E_4 -> E_3 [0.3333]
        E_4 -> E_3 E_3 [0.3333]
        E_4 -> E_3 E_3 E_3 [0.3333]
        E_3 -> E_2 [0.3333]
        E_3 -> E_2 E_2 [0.3333]
        E_3 -> E_2 [0.3333]
        E_2 -> E_1 E_1 [0.3333]
        E_2 -> E_1 [0.3333]
        E_2 -> E_1 E_1 E_1 [0.3333]
        E_1 -> '5' '6' [0.3333]
        E_1 -> '1' '8' '6' '6' [0.3333]
        E_1 -> '1' '5' '1' '5' '5' '9' [

In [3]:
to_latex_equation(grammar_string, color_dict=color_dict, script_notation="_")
# to_latex_equation(grammar_string, script_notation="_")

\begin{align*}
	& \textcolor{red}{S}\;\textcolor{black}{\rightarrow}\;\textcolor{red}{S5}\;\textcolor{blue}{[1]}\\
	& \textcolor{red}{S5}\;\textcolor{black}{\rightarrow}\;\textcolor{red}{B4}\;\textcolor{red}{C1_1}\;\textcolor{red}{E4}\;\textcolor{red}{T1_1}\;\textcolor{blue}{[0.25]}\\
	& \textcolor{red}{S5}\;\textcolor{black}{\rightarrow}\;\textcolor{red}{B4}\;\textcolor{red}{C1_2}\;\textcolor{red}{E4}\;\textcolor{red}{T1_2}\;\textcolor{blue}{[0.25]}\\
	& \textcolor{red}{S5}\;\textcolor{black}{\rightarrow}\;\textcolor{red}{B4}\;\textcolor{red}{C1_3}\;\textcolor{red}{E4}\;\textcolor{red}{T1_3}\;\textcolor{blue}{[0.25]}\\
	& \textcolor{red}{S5}\;\textcolor{black}{\rightarrow}\;\textcolor{red}{B4}\;\textcolor{red}{C1_4}\;\textcolor{red}{E4}\;\textcolor{red}{T1_4}\;\textcolor{blue}{[0.25]}\\
	& \textcolor{red}{B4}\;\textcolor{black}{\rightarrow}\;\textcolor{red}{B3}\;\textcolor{blue}{[0.3333]}\\
	& \textcolor{red}{B4}\;\textcolor{black}{\rightarrow}\;\textcolor{red}{B3}\;\textcolor{red}{B3

## Generation

In [4]:
num_samples = 10000
# num_samples = 10
if grammar_name == "pcfg_cfg3b_disjoint_terminals_skewed_prob":
    num_samples = 50000
seed = 5
sequences, sequence_to_non_terminal_applied_position_map, sequence_freq, sequence_prob_dict = get_grammatical_sentences(grammar, num_samples, seed)

len(sequences), len(sequence_freq), len(sequence_prob_dict), len(sequence_to_non_terminal_applied_position_map)

698it [00:00, 6977.08it/s]

Found sentences with different probabilities! 5.2195660772425e-08 vs 1.7396813735449254e-08


10000it [00:01, 5348.86it/s]


(10000, 9645, 9645, 9645)

In [5]:
grammar_meta_data = {
    "grammar_name": grammar_name,
    "grammar_string": grammar_string,
    "terminals": list(grammar._lexical_index.keys()),
    "num_terminals": len(grammar._lexical_index),
    "num_nonterminals": len(set([production.lhs() for production in grammar.productions()])) if isinstance(grammar, PCFG) else len(grammar.nonterminals),
    "expected_length": np.mean([len(sequence) for sequence in sequences if len(sequence) > 0]),
}

for key in pfsa_object:
    if key in ['rank', 'entropy_analytic']:
        grammar_meta_data[key] = pfsa_object[key]

# frequency
prob_list = list(sequence_freq.values())
sum_prob = sum(prob_list) # normalize
prob_list = [x/sum_prob for x in prob_list]
entropy = 0
for prob in prob_list:
    if prob == 0:
        continue
    entropy += -1 * prob * math.log(prob, 2)

grammar_meta_data['entropy_freq_approximation'] = entropy

# generation probability
prob_list = list(sequence_prob_dict.values())
sum_prob = sum(prob_list) # normalize
prob_list = [x/sum_prob for x in prob_list]
entropy = 0
for prob in prob_list:
    if prob == 0:
        continue
    entropy += -1 * prob * math.log(prob, 2)


grammar_meta_data['entropy_prob_approximation'] = entropy


grammar_meta_data

{'grammar_name': 'pcfg_4_3_1_2_3_4_5_6_7_8_9',
 'grammar_string': "\n        S -> S_5 [1]\n        S_5 -> B_4 C_1_1 E_4 T_1_1 [0.25]\n        S_5 -> B_4 C_1_2 E_4 T_1_2 [0.25]\n        S_5 -> B_4 C_1_3 E_4 T_1_3 [0.25]\n        S_5 -> B_4 C_1_4 E_4 T_1_4 [0.25]\n        B_4 -> B_3 [0.3333]\n        B_4 -> B_3 B_3 B_3 [0.3333]\n        B_4 -> B_3 B_3 [0.3333]\n        B_3 -> B_2 [0.3333]\n        B_3 -> B_2 [0.3333]\n        B_3 -> B_2 B_2 [0.3333]\n        B_2 -> B_1 [0.3333]\n        B_2 -> B_1 [0.3333]\n        B_2 -> B_1 B_1 B_1 [0.3333]\n        B_1 -> '2' '9' '3' [0.3333]\n        B_1 -> '9' '6' '1' [0.3333]\n        B_1 -> '1' '8' '6' '2' [0.3333]\n        E_4 -> E_3 [0.3333]\n        E_4 -> E_3 E_3 [0.3333]\n        E_4 -> E_3 E_3 E_3 [0.3333]\n        E_3 -> E_2 [0.3333]\n        E_3 -> E_2 E_2 [0.3333]\n        E_3 -> E_2 [0.3333]\n        E_2 -> E_1 E_1 [0.3333]\n        E_2 -> E_1 [0.3333]\n        E_2 -> E_1 E_1 E_1 [0.3333]\n        E_1 -> '5' '6' [0.3333]\n        E_1 -> 

In [6]:
# delete empty sequences
sequences = [sequence for sequence in sequences if len(sequence) > 0]
sequence_freq = {sequence: freq for sequence, freq in sequence_freq.items() if len(sequence) > 0}
sequence_prob_dict = {sequence: prob for sequence, prob in sequence_prob_dict.items() if len(sequence) > 0}
sequence_to_non_terminal_applied_position_map = {sequence: non_terminal_applied_position_map for sequence, non_terminal_applied_position_map in sequence_to_non_terminal_applied_position_map.items() if len(sequence) > 0}

len(sequences), len(sequence_freq), len(sequence_prob_dict), len(sequence_to_non_terminal_applied_position_map)

(10000, 9645, 9645, 9645)

In [7]:
show_image = True

In [8]:
# lowest length index
index = 0
length = 1000000
for i in range(len(sequences)):
    if len(sequences[i]) < length:
        length = len(sequences[i])
        index = i

print(index, length)

787 7


In [9]:
fig = plot_nonterminal_map(list(sequences[index]), 
                        sequence_to_non_terminal_applied_position_map[sequences[index]], 
                        # is_hierarchy=grammar_name in grammar_details_dict
                        is_hierarchy=True
                        )

fig.update_layout(
    width=600,
    height=200
)
fig.show()
os.system("mkdir -p ../read_output/figures/sentence_annotated")
store_filename = f"../read_output/figures/sentence_annotated/{grammar_name}_annotated_sentence_{index}.pdf"
fig.write_image(store_filename)
time.sleep(2)
fig.write_image(store_filename)

dict_keys([B_1, B_2, B_3, B_4, C_1_1, E_1, E_2, E_3, E_4, T_1_1, S_5])
{B_1: [(0, 2, (0, B_2, 0))], B_2: [(0, 2, (0, B_3, 1))], B_3: [(0, 2, (0, B_4, 0))], B_4: [(0, 2, (0, S_5, 0))], C_1_1: [(3, 3, (1, S_5, 0))], E_1: [(4, 5, (0, E_2, 1))], E_2: [(4, 5, (0, E_3, 0))], E_3: [(4, 5, (0, E_4, 0))], E_4: [(4, 5, (2, S_5, 0))], T_1_1: [(6, 6, (3, S_5, 0))], S_5: [(0, 6, (0, S, 0))]}


In [10]:
os.system("mkdir -p ../read_output/figures/sentence_annotated")
for i in range(min(10, len(sequences))):
    print(sequences[i])
    if show_image:
        if i <= 5:
            fig = plot_nonterminal_map(list(sequences[i]), 
                                    sequence_to_non_terminal_applied_position_map[sequences[i]], 
                                    # is_hierarchy=grammar_name in grammar_details_dict
                                    # is_hierarchy=False
                                    is_hierarchy=True
                                    )
            fig.show()
            store_filename = f"../read_output/figures/sentence_annotated/{grammar_name}_annotated_sentence_{i}.pdf"
            fig.write_image(store_filename)
            time.sleep(2)
            fig.write_image(store_filename)

meta_data = {
    "non_terminal_applied_position_map": sequence_to_non_terminal_applied_position_map,
    "sequence_freq": sequence_freq,
    "sequence_prob_dict": sequence_prob_dict
}
sequences = sorted(sequences, key=lambda x: len(x[0]))


('1', '8', '6', '2', '2', '9', '3', '9', '6', '1', '9', '6', '1', '1', '8', '6', '2', '2', '9', '3', '9', '6', '1', '7', '1', '5', '1', '5', '5', '9', '1', '5', '1', '5', '5', '9', '3')
dict_keys([B_1, B_2, B_3, B_4, C_1_3, E_1, E_2, E_3, E_4, T_1_3, S_5])
{B_1: [(0, 3, (0, B_2, 2)), (4, 6, (1, B_2, 2)), (7, 9, (2, B_2, 2)), (10, 12, (0, B_2, 2)), (13, 16, (1, B_2, 2)), (17, 19, (2, B_2, 2)), (20, 22, (0, B_2, 0))], B_2: [(0, 9, (0, B_3, 2)), (10, 19, (1, B_3, 2)), (20, 22, (0, B_3, 1))], B_3: [(0, 19, (0, B_4, 2)), (20, 22, (1, B_4, 2))], B_4: [(0, 22, (0, S_5, 2))], C_1_3: [(23, 23, (1, S_5, 2))], E_1: [(24, 29, (0, E_2, 0)), (30, 35, (1, E_2, 0))], E_2: [(24, 35, (0, E_3, 0))], E_3: [(24, 35, (0, E_4, 0))], E_4: [(24, 35, (2, S_5, 2))], T_1_3: [(36, 36, (3, S_5, 2))], S_5: [(0, 36, (0, S, 0))]}


('1', '8', '6', '2', '1', '8', '6', '2', '1', '8', '6', '2', '1', '8', '6', '2', '5', '1', '5', '1', '5', '5', '9', '5', '6', '1', '8', '6', '6', '5', '6', '5', '6', '1', '5', '1', '5', '5', '9', '1', '8', '6', '6', '1', '5', '1', '5', '5', '9', '1', '8', '6', '6', '1')
dict_keys([B_1, B_2, B_3, B_4, C_1_1, E_1, E_2, E_3, E_4, T_1_1, S_5])
{B_1: [(0, 3, (0, B_2, 0)), (4, 7, (0, B_2, 0)), (8, 11, (0, B_2, 0)), (12, 15, (0, B_2, 1))], B_2: [(0, 3, (0, B_3, 0)), (4, 7, (0, B_3, 0)), (8, 11, (0, B_3, 2)), (12, 15, (1, B_3, 2))], B_3: [(0, 3, (0, B_4, 1)), (4, 7, (1, B_4, 1)), (8, 15, (2, B_4, 1))], B_4: [(0, 15, (0, S_5, 0))], C_1_1: [(16, 16, (1, S_5, 0))], E_1: [(17, 22, (0, E_2, 2)), (23, 24, (1, E_2, 2)), (25, 28, (2, E_2, 2)), (29, 30, (0, E_2, 0)), (31, 32, (1, E_2, 0)), (33, 38, (0, E_2, 0)), (39, 42, (1, E_2, 0)), (43, 48, (0, E_2, 0)), (49, 52, (1, E_2, 0))], E_2: [(17, 28, (0, E_3, 2)), (29, 32, (0, E_3, 0)), (33, 42, (0, E_3, 1)), (43, 52, (1, E_3, 1))], E_3: [(17, 28, (0, E_4, 

('1', '8', '6', '2', '2', '9', '3', '1', '8', '6', '2', '9', '6', '1', '7', '1', '5', '1', '5', '5', '9', '5', '6', '3')
dict_keys([B_1, B_2, B_3, B_4, C_1_3, E_1, E_2, E_3, E_4, T_1_3, S_5])
{B_1: [(0, 3, (0, B_2, 0)), (4, 6, (0, B_2, 2)), (7, 10, (1, B_2, 2)), (11, 13, (2, B_2, 2))], B_2: [(0, 3, (0, B_3, 2)), (4, 13, (1, B_3, 2))], B_3: [(0, 13, (0, B_4, 0))], B_4: [(0, 13, (0, S_5, 2))], C_1_3: [(14, 14, (1, S_5, 2))], E_1: [(15, 20, (0, E_2, 0)), (21, 22, (1, E_2, 0))], E_2: [(15, 22, (0, E_3, 0))], E_3: [(15, 22, (0, E_4, 0))], E_4: [(15, 22, (2, S_5, 2))], T_1_3: [(23, 23, (3, S_5, 2))], S_5: [(0, 23, (0, S, 0))]}


('1', '8', '6', '2', '1', '8', '6', '2', '1', '8', '6', '2', '2', '9', '3', '9', '6', '1', '1', '8', '6', '2', '8', '5', '6', '5', '6', '5', '6', '1', '5', '1', '5', '5', '9', '5', '6', '1', '5', '1', '5', '5', '9', '5', '6', '1', '5', '1', '5', '5', '9', '4')
dict_keys([B_1, B_2, B_3, B_4, C_1_4, E_1, E_2, E_3, E_4, T_1_4, S_5])
{B_1: [(0, 3, (0, B_2, 1)), (4, 7, (0, B_2, 2)), (8, 11, (1, B_2, 2)), (12, 14, (2, B_2, 2)), (15, 17, (0, B_2, 0)), (18, 21, (0, B_2, 1))], B_2: [(0, 3, (0, B_3, 1)), (4, 14, (0, B_3, 0)), (15, 17, (0, B_3, 2)), (18, 21, (1, B_3, 2))], B_3: [(0, 3, (0, B_4, 1)), (4, 14, (1, B_4, 1)), (15, 21, (2, B_4, 1))], B_4: [(0, 21, (0, S_5, 3))], C_1_4: [(22, 22, (1, S_5, 3))], E_1: [(23, 24, (0, E_2, 0)), (25, 26, (1, E_2, 0)), (27, 28, (0, E_2, 0)), (29, 34, (1, E_2, 0)), (35, 36, (0, E_2, 0)), (37, 42, (1, E_2, 0)), (43, 44, (0, E_2, 1)), (45, 50, (0, E_2, 1))], E_2: [(23, 26, (0, E_3, 1)), (27, 34, (1, E_3, 1)), (35, 42, (0, E_3, 2)), (43, 44, (0, E_3, 1)), (45, 50,

('1', '8', '6', '2', '1', '8', '6', '2', '9', '6', '1', '1', '8', '6', '2', '5', '5', '6', '1', '8', '6', '6', '1', '5', '1', '5', '5', '9', '1', '8', '6', '6', '1', '8', '6', '6', '1', '5', '1', '5', '5', '9', '1')
dict_keys([B_1, B_2, B_3, B_4, C_1_1, E_1, E_2, E_3, E_4, T_1_1, S_5])
{B_1: [(0, 3, (0, B_2, 1)), (4, 7, (0, B_2, 2)), (8, 10, (1, B_2, 2)), (11, 14, (2, B_2, 2))], B_2: [(0, 3, (0, B_3, 0)), (4, 14, (0, B_3, 0))], B_3: [(0, 3, (0, B_4, 2)), (4, 14, (1, B_4, 2))], B_4: [(0, 14, (0, S_5, 0))], C_1_1: [(15, 15, (1, S_5, 0))], E_1: [(16, 17, (0, E_2, 2)), (18, 21, (1, E_2, 2)), (22, 27, (2, E_2, 2)), (28, 31, (0, E_2, 2)), (32, 35, (1, E_2, 2)), (36, 41, (2, E_2, 2))], E_2: [(16, 27, (0, E_3, 0)), (28, 41, (0, E_3, 2))], E_3: [(16, 27, (0, E_4, 1)), (28, 41, (1, E_4, 1))], E_4: [(16, 41, (2, S_5, 0))], T_1_1: [(42, 42, (3, S_5, 0))], S_5: [(0, 42, (0, S, 0))]}


('1', '8', '6', '2', '9', '6', '1', '9', '6', '1', '6', '1', '5', '1', '5', '5', '9', '1', '8', '6', '6', '1', '5', '1', '5', '5', '9', '2')
dict_keys([B_1, B_2, B_3, B_4, C_1_2, E_1, E_2, E_3, E_4, T_1_2, S_5])
{B_1: [(0, 3, (0, B_2, 2)), (4, 6, (1, B_2, 2)), (7, 9, (2, B_2, 2))], B_2: [(0, 9, (0, B_3, 1))], B_3: [(0, 9, (0, B_4, 0))], B_4: [(0, 9, (0, S_5, 1))], C_1_2: [(10, 10, (1, S_5, 1))], E_1: [(11, 16, (0, E_2, 2)), (17, 20, (1, E_2, 2)), (21, 26, (2, E_2, 2))], E_2: [(11, 26, (0, E_3, 0))], E_3: [(11, 26, (0, E_4, 0))], E_4: [(11, 26, (2, S_5, 1))], T_1_2: [(27, 27, (3, S_5, 1))], S_5: [(0, 27, (0, S, 0))]}


('2', '9', '3', '9', '6', '1', '1', '8', '6', '2', '1', '8', '6', '2', '9', '6', '1', '7', '1', '8', '6', '6', '1', '5', '1', '5', '5', '9', '1', '8', '6', '6', '1', '8', '6', '6', '3')
('1', '8', '6', '2', '1', '8', '6', '2', '9', '6', '1', '2', '9', '3', '9', '6', '1', '1', '8', '6', '2', '1', '8', '6', '2', '2', '9', '3', '1', '8', '6', '2', '9', '6', '1', '6', '1', '8', '6', '6', '1', '5', '1', '5', '5', '9', '5', '6', '1', '8', '6', '6', '1', '8', '6', '6', '1', '8', '6', '6', '5', '6', '1', '8', '6', '6', '2')
('2', '9', '3', '5', '5', '6', '5', '6', '1', '8', '6', '6', '5', '6', '1', '5', '1', '5', '5', '9', '1', '8', '6', '6', '1', '8', '6', '6', '5', '6', '1')
('1', '8', '6', '2', '9', '6', '1', '9', '6', '1', '7', '1', '5', '1', '5', '5', '9', '5', '6', '1', '8', '6', '6', '1', '8', '6', '6', '1', '5', '1', '5', '5', '9', '1', '8', '6', '6', '1', '5', '1', '5', '5', '9', '1', '8', '6', '6', '3')


### Plot length distribution

In [11]:
import plotly.express as px
import plotly.graph_objects as go


# scatter plot of sequence lengths
fig = go.Figure()
len_sequences = [len(sequence) for sequence in sequences]
fig.add_trace(go.Scatter(x=list(range(len(len_sequences))), y=len_sequences, mode='markers'))
fig.update_layout(title="Scatter plot of sequence lengths",
                  xaxis_title="Sequence Index",
                  yaxis_title="Length")
if show_image:
    fig.show()

In [12]:
# generate pdf of length distribution
os.system("mkdir -p ../read_output/figures/grammar")
fig = go.Figure()
fig.add_trace(go.Histogram(x=len_sequences, nbinsx=100, histnorm='probability density'))
fig.update_layout(xaxis_title="Length in Tokens",
                  yaxis_title="Probability")
fig = save_image_template(fig, height=200, width=300)
# if grammar_name.startswith("pcfg_cfg3b_disjoint_terminals"):
#     fig.update_yaxes(range=[0, 0.65])
if show_image:
    fig.show()
store_filename = f"../read_output/figures/grammar/{grammar_name}_length_distribution.pdf"
print(store_filename)
fig.write_image(store_filename)
time.sleep(2)
fig.write_image(store_filename)

../read_output/figures/grammar/pcfg_4_3_1_2_3_4_5_6_7_8_9_length_distribution.pdf


In [13]:
get_stat(sequences)

Number of Sequences: 10000
Unique Sequences: 9645
Max length: 99
Min length: 7
Mean length: 37.8384
Std length: 14.897653689088092


{'num sequences': 10000,
 'unique sequences': 9645,
 'max': 99,
 'min': 7,
 'mean': np.float64(37.8384),
 'std': np.float64(14.897653689088092)}

## Training and test data

In [14]:
train_size = num_samples//2
test_size = num_samples - train_size


# shuffle data
random.seed(seed)
random.shuffle(sequences)

ratio = (train_size) / (train_size + test_size)

non_grammatical_sentence_size = test_size
sequences = sequences[:train_size + test_size]

train_sequences, test_sequences = train_test_split(sequences, seed, sequence_freq, ratio)
data = {
        "train_sequences": train_sequences,
        "test_sequences": test_sequences,
}

Train: 5000
Test: 5000


In [15]:
train_sequence_freq_list = []
for sequence in set(train_sequences):
    train_sequence_freq_list.append(sequence_freq[sequence])

test_sequence_freq_list = []
for sequence in set(test_sequences):
    test_sequence_freq_list.append(sequence_freq[sequence])


fig = go.Figure()
# histogram of sequence lengths
fig.add_trace(go.Histogram(x=train_sequence_freq_list, name="Train"))
fig.add_trace(go.Histogram(x=test_sequence_freq_list, name="Test"))
fig.update_layout(title="Sequence frequency distribution",
                  xaxis_title="Sequence Count",
                  yaxis_title="# Occurrences")
# yscale log
fig.update_yaxes(type="log")
if show_image:
    fig.show()

In [16]:
len(sequence_to_non_terminal_applied_position_map), len(set(train_sequences)), len(set(test_sequences)), len(set(train_sequences).intersection(set(test_sequences)))

(9645, 4828, 4817, 0)

## Randomly sampled data

In [17]:
if True:
    non_grammatical_sequences = random_sentence_generator(
                                        grammar=grammar,
                                        num_samples=non_grammatical_sentence_size,
                                        # num_samples=100,
                                        min_length=min(len_sequences),
                                        max_length=max(len_sequences),
                                        sampled_sequences=set(sequence_freq.keys()),
                                        seed=seed,
                                        timeout=1000,
                                        terminal_freq = compute_terminal_freq(train_sequences + test_sequences)
    )
    data["non_grammatical_sequences"] = non_grammatical_sequences
    get_stat(data["non_grammatical_sequences"])

 20%|█▉        | 4999/25000 [00:12<00:48, 409.06it/s]

Membership query time: 9.52 s
Match with sampled sequence: 0
Number of Sequences: 5000
Unique Sequences: 5000
Max length: 98
Min length: 7
Mean length: 52.2524
Std length: 26.583677214411104


## Grammar perturbation

In [18]:
grammar_edit = False
if grammar_name.startswith("pcfg"):
    grammar_edit = True
    total_runs = 5 # construct 5 different perturbed grammars
    num_grammar_edit_sample_per_run = 200
    verbose=False
    grammar_edit_dict = {}
    if grammar_edit:
        # edit_level = 2
        # edit = 1
        # forced_action = "all"

        for edit_level in [1, 2, 3, 4, 5, 6, 7]:
            for edit in [1, 2, 3, 4][:1]:
                for forced_action in ["insert", "delete", "replace", "all"][-1:]:
                    for run_ids in range(total_runs):
                        print(f"Edit level: {edit_level}, Edit: {edit}, Forced action: {forced_action}")
                        try:
                            grammar_perturbed, perturbation_result = get_perturbed_grammar(grammar, 
                                                                                        grammar_name, 
                                                                                        level=edit_level, 
                                                                                        edit=edit, 
                                                                                        forced_action=forced_action if forced_action != "all" else None,
                                                                                        seed=seed+run_ids, 
                                                                                        verbose=False)


                            grammar_edit_sequences, \
                            grammar_edit_sequence_to_non_terminal_applied_position_map, \
                            grammar_edit_sequence_freq, \
                            grammar_edit_sequence_prob_dict = get_nongrammatical_sentences_from_perturbed_grammar(
                                                            base_grammar=grammar,
                                                            perturbed_grammar=grammar_perturbed, 
                                                            num_samples=num_grammar_edit_sample_per_run, 
                                                            seed=seed)
                            
                            if len(grammar_edit_sequences) == 0:
                                continue

                            if f"non_grammatical_test_sequences_grammar_edit_{edit_level}_{edit}_{forced_action}" not in data:
                                data[f"non_grammatical_test_sequences_grammar_edit_{edit_level}_{edit}_{forced_action}"] = []
                            data[f"non_grammatical_test_sequences_grammar_edit_{edit_level}_{edit}_{forced_action}"] += grammar_edit_sequences
                            if f"grammar_edit_{edit_level}_{edit}_{forced_action}" not in grammar_edit_dict:
                                grammar_edit_dict[f"grammar_edit_{edit_level}_{edit}_{forced_action}"] = []    
                            grammar_edit_dict[f"grammar_edit_{edit_level}_{edit}_{forced_action}"].append({
                                "base_grammar": grammar,
                                "perturbed_grammar": grammar_perturbed,
                                "perturbation_result": perturbation_result,
                                "sequences": grammar_edit_sequences,
                                "non_terminal_applied_position_map": grammar_edit_sequence_to_non_terminal_applied_position_map,
                                "sequence_freq": grammar_edit_sequence_freq,
                                "sequence_prob_dict": grammar_edit_sequence_prob_dict,
                                "edit_level": edit_level,
                                "edit": edit,
                                "forced_action": forced_action
                            })


                            print(f"Generated sequences: {len(grammar_edit_sequences)}")

                            if verbose:
                                for nonterminal in perturbation_result:
                                    print(f"{nonterminal}:")
                                    for i, (production_before, production_after) in enumerate(zip(grammar.productions(nonterminal), grammar_perturbed.productions(nonterminal))):
                                        if i not in perturbation_result[nonterminal]:
                                            continue
                                        # print(f"{i}: {production_before.rhs()} => {production_after.rhs()}")
                                        print(f"{production_before.rhs()} => {production_after.rhs()}")
                                        print(perturbation_result[nonterminal][i])
                                        print()

                                if show_image:
                                    for index in range(min(1, len(sequences))):
                                        plot_nonterminal_map_with_edit(
                                            token_sequence=list(grammar_edit_sequences[index]),
                                            nonterminal_applied_position_map=grammar_edit_sequence_to_non_terminal_applied_position_map[grammar_edit_sequences[index]],
                                            grammar_perturbed=grammar_perturbed,
                                            perturbation_result=perturbation_result,
                                            edit_level=edit_level-1,
                                            verbose=False,
                                        ).show()
                        except:
                            pass

Edit level: 1, Edit: 1, Forced action: all


236it [00:01, 133.41it/s]


Generated sequences: 200
Edit level: 1, Edit: 1, Forced action: all


833it [00:12, 66.17it/s]


Generated sequences: 200
Edit level: 1, Edit: 1, Forced action: all


...

Generated sequences: 200
Edit level: 5, Edit: 1, Forced action: all


884it [00:13, 63.68it/s]


Generated sequences: 200
Edit level: 5, Edit: 1, Forced action: all


884it [00:13, 63.25it/s]

Generated sequences: 200
Edit level: 6, Edit: 1, Forced action: all
Edit level: 6, Edit: 1, Forced action: all
Edit level: 6, Edit: 1, Forced action: all
Edit level: 6, Edit: 1, Forced action: all
Edit level: 6, Edit: 1, Forced action: all
Edit level: 7, Edit: 1, Forced action: all
Edit level: 7, Edit: 1, Forced action: all
Edit level: 7, Edit: 1, Forced action: all
Edit level: 7, Edit: 1, Forced action: all
Edit level: 7, Edit: 1, Forced action: all


## Edit distance (lexer)

In [19]:
edit_distance_lexer = True
if edit_distance_lexer:
    edit_distance_perturb_dict = {}
    edit_distance_non_terminal_mapping = {}

    # for perturb_start_index, perturb_end_index in [(0, 25), (25, 50), (50, 200)]:
    # for perturb_start_index, perturb_end_index in [(0, 40), (40, 100), (100, 1000)]:
    stat = get_stat(test_sequences)
    for perturb_start_index, perturb_end_index in [(stat['min'], stat['max'])]:
        if perturb_start_index == perturb_end_index:
            print("Min and max length are same")
            perturb_start_index = 1
        for edit_distance in [1, 2, 3]:
            # test
            perturbed_sequences, perturb_position_dict, perturbed_sequence_to_non_terminal_applied_position_map = \
                    generate_similar_sequences(grammar, 
                                            test_sequences, 
                                            sequence_to_non_terminal_applied_position_map,
                                            edit_distance, 
                                            seed,
                                            sampled_sequences=set(sequence_freq.keys()), 
                                            perturb_start_index=perturb_start_index, 
                                            perturb_end_index=perturb_end_index)
            data[f"non_grammatical_test_sequences_edit_distance_{edit_distance}_{perturb_start_index}_{perturb_end_index}"] = perturbed_sequences
            edit_distance_perturb_dict[f"non_grammatical_test_sequences_edit_distance_{edit_distance}_{perturb_start_index}_{perturb_end_index}"] = perturb_position_dict
            edit_distance_non_terminal_mapping[f"non_grammatical_test_sequences_edit_distance_{edit_distance}_{perturb_start_index}_{perturb_end_index}"] = perturbed_sequence_to_non_terminal_applied_position_map


    stat = get_stat(train_sequences)
    for perturb_start_index, perturb_end_index in [(stat['min'], stat['max'])]:
        if perturb_start_index == perturb_end_index:
            print("Min and max length are same")
            perturb_start_index = 1
        for edit_distance in [1, 2, 3]:
            # train
            perturbed_sequences, perturb_position_dict, perturbed_sequence_to_non_terminal_applied_position_map = \
                    generate_similar_sequences(grammar, 
                                            train_sequences, 
                                            sequence_to_non_terminal_applied_position_map,
                                            edit_distance, 
                                            seed,
                                            sampled_sequences=set(sequence_freq.keys()), 
                                            perturb_start_index=perturb_start_index, 
                                            perturb_end_index=perturb_end_index)
            data[f"non_grammatical_train_sequences_edit_distance_{edit_distance}_{perturb_start_index}_{perturb_end_index}"] = perturbed_sequences
            edit_distance_perturb_dict[f"non_grammatical_train_sequences_edit_distance_{edit_distance}_{perturb_start_index}_{perturb_end_index}"] = perturb_position_dict
            edit_distance_non_terminal_mapping[f"non_grammatical_train_sequences_edit_distance_{edit_distance}_{perturb_start_index}_{perturb_end_index}"] = perturbed_sequence_to_non_terminal_applied_position_map

    meta_data_edit_distance = {
        "non_terminal_applied_position_map": edit_distance_non_terminal_mapping,
        "perturbation_result": edit_distance_perturb_dict
    }            

Number of Sequences: 5000
Unique Sequences: 4817
Max length: 94
Min length: 7
Mean length: 37.6012
Std length: 14.878405780190297


100%|██████████| 5000/5000 [00:37<00:00, 134.95it/s]


21.665129336274312 12.023344431052273
Match with sampled sequence: 0


100%|██████████| 5000/5000 [00:28<00:00, 176.39it/s]


21.71838331160365 12.158636276268165
Match with sampled sequence: 0


...

22.270194427741032 12.51630148040256
Match with sampled sequence: 0


100%|██████████| 5000/5000 [00:30<00:00, 165.98it/s]


21.953092111857273 12.356154587494935
Match with sampled sequence: 0


100%|██████████| 5000/5000 [00:24<00:00, 204.31it/s]

21.92702485966319 12.306677500029492
Match with sampled sequence: 0


In [20]:
for key in data:
    print(key, len(data[key]), len(set(data[key])))
    # get_stat(data[key])
    # for sequence in data[key][:5]:
    #     print(sequence)
    # print()


train_sequences 5000 4828
test_sequences 5000 4817
non_grammatical_sequences 5000 5000
non_grammatical_test_sequences_grammar_edit_1_1_all 800 800
non_grammatical_test_sequences_grammar_edit_2_1_all 600 598
non_grammatical_test_sequences_grammar_edit_3_1_all 800 599
non_grammatical_test_sequences_grammar_edit_4_1_all 600 597
non_grammatical_test_sequences_grammar_edit_5_1_all 1000 951
non_grammatical_test_sequences_edit_distance_1_7_94 4987 4978
non_grammatical_test_sequences_edit_distance_2_7_94 4948 4942
non_grammatical_test_sequences_edit_distance_3_7_94 4986 4978
non_grammatical_train_sequences_edit_distance_1_7_99 4989 4982
non_grammatical_train_sequences_edit_distance_2_7_99 4939 4939
non_grammatical_train_sequences_edit_distance_3_7_99 4987 4984


In [21]:
g2 = grammar_name
if os.path.isfile(f"../data_backup/{g2}/sequences_w_edit_distance_{g2}_10000_5.pkl"):
    with open(f"../data_backup/{g2}/sequences_w_edit_distance_{g2}_10000_5.pkl", 'rb') as f:
        data_g2 = pickle.load(f)
        # print(data_g2.keys()) 
        for key in data_g2:
            # print(key)
            assert key in data
            print(key, len(data_g2[key]), len(set(data_g2[key])))
            for i, sequence in enumerate(data_g2[key]):
                assert sequence == data[key][i]
            

## Store

In [22]:
store_path = "../data"
os.makedirs(f"{store_path}/{grammar_name}", exist_ok=True)
filename = f"{store_path}/{grammar_name}/sequences_w_edit_distance_{grammar_name}_{train_size + test_size}_{seed}.pkl"
with open(filename, 'wb') as f:
    pickle.dump(data, f)

In [23]:
assert num_samples == train_size + test_size
filename = f"{store_path}/{grammar_name}/meta_data_{grammar_name}_{num_samples}_{seed}.pkl"
with open(filename, 'wb') as f:
    pickle.dump(meta_data, f)

In [24]:
if edit_distance_lexer:
    filename = f"{store_path}/{grammar_name}/meta_data_lexer_edit_{grammar_name}_{num_samples}_{seed}.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(meta_data_edit_distance, f)

In [25]:
if grammar_edit:
    filename = f"{store_path}/{grammar_name}/meta_data_grammar_edit_{grammar_name}_{num_samples}_{seed}.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(grammar_edit_dict, f)

In [26]:
filename = f"{store_path}/{grammar_name}/grammar_meta_data_{grammar_name}.pkl"
with open(filename, 'wb') as f:
    pickle.dump(grammar_meta_data, f)

In [27]:
grammar_name

'pcfg_4_3_1_2_3_4_5_6_7_8_9'